In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi as π

# Laplace's equation

In this demo I'll show how to solve Laplace's equation with firedrake, as well as a few other handy things along the way.
Firedrake makes it possible to specify the weak form of the PDE you're solving symbolically.
It will generate efficient C code to fill the finite element matrices and vectors so you don't have to.

In [ ]:
import firedrake

### Input data

First we need to define the spatial domain.
I used the mesh generator [gmsh](https://gmsh.info) to do this, but you can also use Triangle, Exodus, etc.
Firedrake can read the output directly.

In [ ]:
mesh = firedrake.Mesh('domain.msh')

The domain with a legend for different parts of the boundary.
The numeric IDs are used to set where you want different boundary conditions.

In [ ]:
fig, axes = plt.subplots()
firedrake.plot(mesh, axes=axes)

Next we have to decide which basis functions to use for our problem.
Here we'll use continuous Galerkin (CG) elements with polynomial degree 1.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=1)

Now we have to decide on what the right-hand side of the PDE will be.
We can define it from an algebraic expression.
To do this, we get some symbolic handles `x`, `y` that represent the coordinates of any point in the domain.

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)

Next, we can make an expression describing what we want the right-hand side to be.
All of the usual algebraic operations (`+`, `-`, `*`, `/`) have been overloaded for these symbolic objects, and firedrake has its own symbolic transcendental functions (`sin`, `cos`, `exp`, etc.).

In [ ]:
k, l = 2, 1
ϕ = π * (k * x + l * y)
expr = firedrake.sin(ϕ)

Note how forming expressions is *composable* -- we can first form an expression for the phase $\phi$, and then pass this to the `sin` function.
We could have done this all in one line of code of course, but it's really handy to be able to break up complex expressions into smaller ones.
Embedded calculators like the field calculator in Paraview or QGIS don't have this feature!

We can evaluate this expression at a point by passing in the coordinates as a tuple.

In [ ]:
print(ϕ((0.2, -0.1)))

The variable `expr` represents an algebraic expression.
To get a finite element field out of it, we call the `interpolate` function.
We also have to pass in the function space we'll be interpolating the expression to.

In [ ]:
f = firedrake.interpolate(expr, Q)

### Analyzing functions

Before we look at how to specify a PDE let's see how to calculate a few things about functions.
The first thing you'll want to do is make plots of them.

In [ ]:
axes = firedrake.plot(f, cmap='viridis')
axes.set_aspect('equal')

But often you'll want to calculate functionals of the input data to a PDE or the solution.
We can use the same algebraic or symbolic tools to define these functionals.
For example, the following functional represents the squared $L^2$-norm of the right-hand side $f$:

In [ ]:
from firedrake import dx
norm_squared = f**2 * dx

The variable `dx` represents integration over the entire domain.
The variable `norm_squared` is not yet a number -- it's a symbolic entity that describes how to compute a number from the input fields, which in this case are just `f`.

In [ ]:
print(type(norm_squared))

To actually compute the number that this quantity represents, we pass it to the function `assemble`.
The `assemble` function ultimately does all the real work of looping over the cells of a triangulation with some quadrature rule.

In [ ]:
print(firedrake.assemble(norm_squared))

If we change the value of the function `f` by assigning it some new expression, assembling `norm_squared` again will give a different value.

In [ ]:
k, l = 4, 6
ϕ = π * (k * x + l * y)
new_expr = 0.25 * firedrake.cos(ϕ)
f += firedrake.interpolate(new_expr, Q)

In [ ]:
print(firedrake.assemble(norm_squared))

In [ ]:
axes = firedrake.plot(f, cmap='viridis')
axes.set_aspect('equal')

We can also evaluate integrals over the boundary of the domain by using a different measure.

In [ ]:
from firedrake import ds
print(firedrake.assemble(f**2 * ds))

So far, we've just looked at integrals of `f`, but we can also look at its gradient.

In [ ]:
from firedrake import inner, grad
energy = inner(grad(f), grad(f)) * dx
print(firedrake.assemble(energy))

The function `inner` calculates the inner product of two vectors.
Just like how we got a symbolic representation of the mesh coordinates, we can also get a symbol for the outward normal vector to calculate things like boundary fluxes.

In [ ]:
n = firedrake.FacetNormal(mesh)
flux = inner(grad(f), n) * ds
print(firedrake.assemble(flux))

You can calculate the volume of the domain and the area of the boundary by integrating the constant function $1$ times a measure, but you have to specify the mesh otherwise `assemble` won't know what the domain is.

In [ ]:
from firedrake import Constant
volume = firedrake.assemble(Constant(1) * dx(mesh))
perimeter = firedrake.assemble(Constant(1) * ds(mesh))
print('Volume, perimeter: {}, {}'.format(volume, perimeter))

### Defining a PDE

Given a function $f$ in $L^2$, the weak form of the Laplace equation with Dirichlet boundary conditions is

$$\int_\Omega\nabla u\cdot\nabla v\hspace{2pt}dx = \int_\Omega fv\hspace{2pt}d x$$

for all $v$ such that $v|_{\partial\Omega} = 0$.
The functions $v$ are referred to as *test functions*, while the basis functions used to represent $u$ are called *trial functions*.
The following code creates test and trial functions on the space $Q$.

In [ ]:
u = firedrake.TrialFunction(Q)
v = firedrake.TestFunction(Q)

These are again symbolic objects with no actual values.
We use them only to create the weak form of the PDE, which we can do using the algebraic language shown above.

In [ ]:
a = inner(grad(u), grad(v)) * dx

The right-hand side will be quite similar.

In [ ]:
F = f * v * dx

Next we need to add some boundary conditions.
The `'on_boundary'` argument can be replaced with a number of a list of numbers describing the parts of the boundary where Dirichlet conditions apply.

In [ ]:
bc = firedrake.DirichletBC(Q, 0, 'on_boundary')

Finally we can solve the PDE.
We need to create a function in the space $Q$ that will hold the value of the solution.
This is an actual function with an array of finite element expansion coefficients, not a symbolic object like the test and trial functions from above.

In [ ]:
q = firedrake.Function(Q)

Finally, to solve the PDE, we specify the equation in terms of the bilinear form and right-hand side, where we're going to put the solution, the boundary conditions, and how to solve it.

In [ ]:
firedrake.solve(a == F, q, bc,
                solver_parameters={'ksp_type': 'cg',
                                   'pc_type': 'jacobi'})

One important thing to note is that firedrake makes it easy to specify *what problem you are solving*, but it is up to you to decide *how you are going to solve it*.
It is up to you to know that, for example, the Laplace equation is symmetric and positive-definite and that you can then use the conjugate gradient method, as we did here.

In [ ]:
firedrake.plot(q, plot3d=True, cmap='viridis')

In [ ]:
energy = inner(grad(q), grad(q)) * dx
print(firedrake.assemble(energy))

### Try it yourself

Some things you can try:

* Add in non-zero boundary values.
* Use Robin or Neumann boundary conditions.
* Make the input data a random trigonometric polynomial.
* Add a spatially variable conductivity coefficient.
* Use higher-order finite elements.
* Estimate the smallest eigenfunction + eigenvalue via the inverse power method.